## Assignement 5
Lukas Schreiner, Asse Reinhardus, Max Tragl

In [1]:
#Import Libraries
import numpy as np
import scipy.stats as st

### Question 1.1

In [2]:
interest = (100-98)/100

n = (5 - 0) / (65 - 45)
m = (0 - 45*n) / (1 + interest)

C = np.round(n * 50 + m * 1,2)
print("The European Call has a price of: ",C)

The European Call has a price of:  1.47


### Question 1.2
Yes you could replicate the call option with bonds and the underlying such that the payoff of the call and the portfolio are the same. Then you buy this portfolio witch should have a value of 1.47 and short the call which is trading at 2. Your payoff at maturity is zero but you make a riskless profit of (2 - 1.47 = 0.53)

### Question 1.3


In [3]:
n = (0 - 15) / (65 - 45)
m = (15 - 45*n) / (1 + interest)

P = np.round(n * 50 + m * 1,2)
print("The European Put should have a price of: ",P)

The European Put should have a price of:  10.29


In [4]:
#Call price plus discounted strike 
print(np.round(C + 60/1.02,2))

#Put price plus current price of the underlying 
print(P + 50)
print("Put Call Parity is satisfied if the put would have been correctly priced")

60.29
60.29
Put Call Parity is satisfied if the put would have been correctly priced


### Question 2.1

The red line describes a call option with strike K = 30 and maturity one year <br>
The dark blue line describes a call option with strike K = 60 and maturity one year <br>
The light blue line describes a call option with strike K = 60 and maturity 6 months year <br>

The purple line describes a put option with strike K = 30 and maturity one year <br>
The yellow line describes a put option with strike K = 60 and maturity one year <br>
The grey blue line describes a put option with strike K = 60 and maturity 6 months year <br>


The reason for that is that call options exhibit a delta between 0 and 1 (option price rises is the price of the underlying rises), whereas  put options have a delta bewteen 0 and -1 (option prices goes down as the underlying rises). 
As we approach maturity the probability of getting from out-of-the-money into in-of-the-money and vice versa gets lower. For that reason more of the in-of-the-money the money options tend to exhibit the same price movments as the underlying which leads to to a steeper slope of the delta curve. 

### Question 2

In [5]:
def black_scholes(S, K, T, r, sigma, opt_type):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    #opt_type: type of option 
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * st.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * st.norm.cdf(d2, 0.0, 1.0))
    put = (K * np.exp(-r * T) * st.norm.cdf(-d2, 0.0, 1.0) - S * st.norm.cdf(-d1, 0.0, 1.0))
    
    delta_call = st.norm.cdf(d1, 0.0, 1.0)
    delta_put = -st.norm.cdf(-d1, 0.0, 1.0)
    
    if opt_type.upper() == 'CALL':
        return np.round(call,4), np.round(delta_call,4)
    
    if opt_type.upper() == 'PUT':
        return np.round(put,4), np.round(delta_put,4)


### Question 2.2

In [6]:
#Strategy 1
sp500_price = 2740
sp500_sigma = 0.18
interest_rate_a = 0.02

opt1a_delta = black_scholes(sp500_price, 2740, 3/12, interest_rate_a, sp500_sigma,'Call')[1]
opt2a_delta = black_scholes(sp500_price, 2300, 6/12, interest_rate_a, sp500_sigma,'Call')[1]
opt3a_delta = black_scholes(sp500_price, 2990, 9/12, interest_rate_a, sp500_sigma,'Call')[1]

print("Option 1 Delta: ", opt1a_delta)
print("Option 2 Delta: ", opt2a_delta)
print("Option 3 Delta: ", opt3a_delta)

strategya_delta = 100 * opt1a_delta + 200 * opt2a_delta + 50 * opt3a_delta
print("Strategy Delta ", strategya_delta)

Option 1 Delta:  0.54
Option 2 Delta:  0.9354
Option 3 Delta:  0.3498
Strategy Delta  258.57


In [7]:
#Strategy 2
SMI_price = 8900
SMI_sigma = 0.18
interest_rate_b = 0.02

opt1b_delta = black_scholes(SMI_price, 8990, 1/12, interest_rate_b, SMI_sigma,'Put')[1]
opt2b_delta = black_scholes(SMI_price, 8500, 6/12, interest_rate_b, SMI_sigma,'Call')[1]
opt3b_delta = black_scholes(SMI_price, 8300, 12/12, interest_rate_b, SMI_sigma,'Put')[1]

print("Option 1 Delta: ", opt1b_delta)
print("Option 2 Delta: ", opt2b_delta)
print("Option 3 Delta: ", opt3b_delta)

strategyb_delta = 1 * opt1b_delta + 2 * opt2b_delta + 1 * opt3b_delta
print("Strategy Delta ", strategyb_delta)

Option 1 Delta:  -0.5539
Option 2 Delta:  0.6927
Option 3 Delta:  -0.278
Strategy Delta  0.5535


In [8]:
#Strategy 3
DAX_price = 109000
DAX_sigma = 0.12
interest_rate_c = 0.05

opt1c_delta = black_scholes(DAX_price, 109000, 6/12, interest_rate_c, DAX_sigma,'Put')[1]
opt2c_delta = black_scholes(DAX_price, 109000, 6/12, interest_rate_c, DAX_sigma,'Call')[1]


print("Option 1 Delta: ", opt1c_delta)
print("Option 2 Delta: ", opt2c_delta)

strategyc_delta = 1 * opt1c_delta + 1 * opt2c_delta
print("Strategy Delta ", strategyc_delta)

Option 1 Delta:  -0.368
Option 2 Delta:  0.632
Strategy Delta  0.264


### Question 3.1

In [9]:
#Parameters
strike = 2000
S_0 = 2000
h = 1/6

In [10]:
def create_tree_underlying(vol):
    
    u = np.exp(np.sqrt(vol*h))
    d = 1/u

    def add_tree_layer(input_layer):
        results = []
        
        #Up
        [results.append(values * u) for values in input_layer]

        #Down
        results.append(input_layer[0] * d)

        return list(np.round(np.sort(results),2))
    
    tree_data = []
    #Period 0
    tree_data.append(S_0)
    #Period 1
    tree_data.append(np.round([S_0 * d, S_0 * u],2))

    #Periods 2 to 6
    for i in range(1,6):
        tree_data.append(add_tree_layer(tree_data[i]))
        
    [print(tree_data[i]) for i in range(0,7)] 
    
    return tree_data

In [11]:
tree_data1 = create_tree_underlying(0.2)

2000
[1666.25 2400.61]
[1388.19, 2000.01, 2881.46]
[1156.53, 1666.25, 2400.62, 3458.63]
[963.53, 1388.19, 2000.01, 2881.47, 4151.4]
[802.74, 1156.53, 1666.25, 2400.62, 3458.64, 4982.94]
[668.78, 963.53, 1388.19, 2000.01, 2881.47, 4151.42, 5981.04]


### Question 3.1

In [12]:
def create_tree_derivative_pricing(vol, tree_data):
    
    u = np.exp(vol*np.sqrt(h))
    d = 1/u
     
    #Risk Neutral Probabilities
    q = np.round((np.exp(0) - d) / (u - d),4)
    print("Risk Neutral Probability (Up): ", q)
    
    #Find Payoffs at the final node
    payoff_layer = np.round(np.log(np.divide(strike,tree_data[-1])),4)
    
    #Backpropageagtion Algorithm
    #Calculate the expected Values by with the risk neutral probabilities per layer
    def add_tree_layer_future(input_layer):

        results = []

        [results.append(input_layer[i]*(1 - q) + input_layer[i+1] * q) for i in range(0,len(input_layer) - 1)]

        return list(np.round(results,4))
    
    #Initilialize the value of the future tree
    tree_data_future = []
    #First Iteration
    tree_data_future.append(add_tree_layer_future(payoff_layer))
    #Subsequent Iterations
    for i in range(0,5):
        tree_data_future.append(add_tree_layer_future(tree_data_future[i]))
        
    [print(tree_data_future[i]) for i in range(0,len(tree_data_future))]
    
    print("Future Price is: ", tree_data_future[-1][0])
    

In [13]:
create_tree_derivative_pricing(0.2, tree_data1)

Risk Neutral Probability (Up):  0.4796
[0.9203, 0.5552, 0.19, -0.1751, -0.5403, -0.9054]
[0.7452, 0.3801, 0.0149, -0.3502, -0.7154]
[0.5701, 0.205, -0.1602, -0.5253]
[0.395, 0.0299, -0.3353]
[0.2199, -0.1452]
[0.0448]
Future Price is:  0.0448


### Question 3.3.1

In [14]:
tree_data2 = create_tree_underlying(0.25)

2000
[1630.72 2452.9 ]
[1329.63, 2000.0, 3008.36]
[1084.13, 1630.73, 2452.9, 3689.6]
[883.96, 1329.63, 2000.01, 3008.36, 4525.11]
[720.75, 1084.13, 1630.73, 2452.91, 3689.6, 5549.82]
[587.67, 883.96, 1329.63, 2000.01, 3008.37, 4525.11, 6806.58]


In [15]:
create_tree_derivative_pricing(0.25, tree_data2)

Risk Neutral Probability (Up):  0.4745
[1.031, 0.6228, 0.2145, -0.1937, -0.602, -1.0102]
[0.8373, 0.4291, 0.0208, -0.3874, -0.7957]
[0.6436, 0.2354, -0.1729, -0.5811]
[0.4499, 0.0417, -0.3666]
[0.2562, -0.152]
[0.0625]
Future Price is:  0.0625


### Question 3.3.2

In [16]:
tree_data3 = create_tree_underlying(0.3)

2000
[1599.26 2501.16]
[1278.82, 2000.0, 3127.9]
[1022.58, 1599.27, 2501.16, 3911.69]
[817.69, 1278.82, 2000.01, 3127.9, 4891.88]
[653.85, 1022.59, 1599.27, 2501.17, 3911.69, 6117.68]
[522.84, 817.69, 1278.83, 2000.01, 3127.91, 4891.88, 7650.64]


In [17]:
create_tree_derivative_pricing(0.3, tree_data3)

Risk Neutral Probability (Up):  0.4694
[1.1317, 0.6845, 0.2373, -0.2099, -0.6571, -1.1043]
[0.9218, 0.4746, 0.0274, -0.4198, -0.867]
[0.7119, 0.2647, -0.1825, -0.6297]
[0.502, 0.0548, -0.3924]
[0.2921, -0.1551]
[0.0822]
Future Price is:  0.0822


### Question 3.4
The higher the volatility, the higher the price of the derivative. The reason for that is that the probability of ending up in the money and receiving a (high) payoff rises as the volatility increases. 